In [1]:
%pip install lightfm

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
import pandas as pd
from scipy.sparse import csr_matrix

BASE_PATH = ''

In [3]:
# Загрузка и обработка данных
users = pd.read_csv(BASE_PATH + "small_player_starts_train.csv")
# users = users.sample(frac=0.01, random_state=42)  # Обрезаем датасет (только для демонстрации)
users['user_id_num'] = users['user_id'].str.split('_').str.get(1).astype(int)
users['item_id_num'] = users['item_id'].str.split('_').str.get(1).astype(int)

# Создаем разреженную матрицу
user_ids = users['user_id_num'].values
item_ids = users['item_id_num'].values

# Предполагая, что у вас есть столбец с рейтингами/взаимодействиями
# Если его нет, можно просто использовать np.ones_like(user_ids)
ratings = np.ones_like(user_ids)

data_crosstab = csr_matrix((ratings, (user_ids, item_ids)))

In [4]:
# У вас уже есть разреженная матрица взаимодействий, поэтому нет необходимости создавать ее снова.
interactions_csr = data_crosstab

# Предположим, что у вас есть список уникальных user_id и item_id, которые вы использовали при создании data_crosstab
unique_user_ids = np.unique(users['user_id_num'])
unique_item_ids = np.unique(users['item_id_num'])

# Создание отображений
user_id_mapping = {real_id: idx for idx, real_id in enumerate(unique_user_ids)}
item_id_mapping = {real_id: idx for idx, real_id in enumerate(unique_item_ids)}

# Предварительная генерация массива для быстрого поиска
item_lookup_array = np.empty(interactions_csr.shape[1], dtype=np.int64)
for real_id, idx in item_id_mapping.items():
    item_lookup_array[idx] = real_id


In [ ]:
# import logging
# from lightfm.evaluation import precision_at_k
# 
# # Настройка логгирования
# logging.basicConfig(level=logging.INFO)
# 
# # Параметры модели и обучения
# k = 5
# epochs = 10
# num_threads = 4
# 
# # Инициализация модели
# model = LightFM(no_components=20, loss='bpr', k=k)
# 
# # Обучение модели с логгированием
# for epoch in range(epochs):
#     logging.info(f"Starting epoch {epoch + 1}")
#     model.fit_partial(interactions_csr, epochs=1, num_threads=num_threads)  # Обучение на одной эпохе
#     precision = precision_at_k(model, interactions_csr, k=k).mean()
#     logging.info(f"Completed epoch {epoch + 1}, Precision at {k}: {precision}")

INFO:root:Starting epoch 1


In [ ]:
# Обучение модели
model = LightFM(no_components=20, loss='warp', k=5)
model.fit(interactions_csr, epochs=10, num_threads=4)

In [ ]:
# Оценка модели
train_precision = precision_at_k(model, interactions_csr, k=5).mean()
print(f'Train precision at k=5: {train_precision}')

In [ ]:
videos = pd.read_parquet(BASE_PATH + "videos.parquet")
videos_t = videos.sort_values(by='channel_sub', ascending=False)
top_videos = videos_t['video_id'].values[:100]
# Функция рекомендации
def recommend_for_user_fast(real_user_id):
    internal_user_id = user_id_mapping.get(real_user_id)
    if internal_user_id is None:
        print(f"User {real_user_id} is not in the data.")
        top_videos = np.random.choice(top_videos, size=10, replace=False)
        return top_videos

    n_users, n_items = interactions_csr.shape
    scores = model.predict(internal_user_id, np.arange(n_items))
    top_items = np.argsort(-scores)[:10]

    top_real_item_ids = item_lookup_array[top_items]

    return top_real_item_ids

In [24]:
real_user_id = 5030883 # Замените на реальный user_id из вашего датасета
recommended_item_ids = recommend_for_user_fast(real_user_id)
print(f"Recommended items for user {real_user_id}: {recommended_item_ids}")



Recommended items for user 5030883: [ 283933   68646 1122625  836422 1508623]


In [ ]:
# Открытие файла sample_submission.csv
submission = pd.read_csv(BASE_PATH + "sample_submission.csv")

# Создаем пустой список для хранения рекомендаций
recommendations = []

# В цикле берем значения user_id из submission и вызываем функцию recommend_for_user_fast
for user_id in submission['user_id']:
    recs = recommend_for_user_fast(user_id)
    # Преобразуем рекомендации в строку, разделенную пробелами (или другим разделителем по вашему выбору)
    recs_str = " ".join(map(str, recs))
    recommendations.append(recs_str)

# Записываем результат в колонку recs в submission
submission['recs'] = recommendations

# Сохраняем измененный DataFrame в новый CSV-файл
submission.to_csv(BASE_PATH + "new_sample_submission.csv", index=False)


In [22]:
users.head()

,date,user_id,item_id,watch_time,is_autorized,user_id_num,item_id_num
558731,2023-08-21 23:27:33+03:00,user_17524578,video_169885,40,1,17524578,169885
1513034,2023-08-21 10:10:41+03:00,user_5030883,video_1122625,2,0,5030883,1122625
404058,2023-08-21 18:41:15+03:00,user_19163580,video_1546491,1,0,19163580,1546491
1249443,2023-08-21 01:20:33+03:00,user_3710106,video_1692997,2,0,3710106,1692997
540190,2023-08-21 21:53:50+03:00,user_25875618,video_1577488,151,0,25875618,1577488
